In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from scipy.stats import gamma
import numpy as np
import c3aidatalake

In [110]:
df = pd.read_csv('raw_counties_info.csv')
df

,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,populationOfAllChildren,latestLaborForce,latestEmployedPopulation,latestUnemployedPopulation,latestUnemploymentRate,laborForceOfAllChildren,locationType,populationCDS,id,name,typeIdent,location.value.id,location.timestamp,fips.id
0,6.0,25.0,25.0,24527.0,24527.0,10090.0,9633.0,457.0,4.529237,10090.0,county,24527.0,Abbeville_SouthCarolina_UnitedStates,Abbeville,EP_LOC,Abbeville_SouthCarolina_UnitedStates,2020-11-14T00:00:00Z,45001.0
1,7.0,171.0,186.0,62045.0,62045.0,24074.0,22369.0,1705.0,7.082329,24074.0,county,62045.0,Acadia_Louisiana_UnitedStates,Acadia,EP_LOC,Acadia_Louisiana_UnitedStates,2020-11-14T00:00:00Z,22001.0
2,10.0,46.0,52.0,32316.0,32316.0,16001.0,15096.0,905.0,5.655897,16001.0,county,32316.0,Accomack_Virginia_UnitedStates,Accomack,EP_LOC,Accomack_Virginia_UnitedStates,2020-11-14T00:00:00Z,51001.0
3,97.0,1195.0,1305.0,481587.0,481587.0,263531.0,248015.0,15516.0,5.887732,263531.0,county,481587.0,Ada_Idaho_UnitedStates,Ada,EP_LOC,Ada_Idaho_UnitedStates,2020-11-14T00:00:00Z,16001.0
4,4.0,25.0,25.0,7152.0,7152.0,3862.0,3747.0,115.0,2.977732,3862.0,county,7152.0,Adair_Iowa_UnitedStates,Adair,EP_LOC,Adair_Iowa_UnitedStates,2020-11-14T00:00:00Z,19001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,42.0,447.0,456.0,213787.0,213787.0,97557.0,80938.0,16619.0,17.035169,97557.0,county,213787.0,Yuma_Arizona_UnitedStates,Yuma,EP_LOC,Yuma_Arizona_UnitedStates,2020-11-14T00:00:00Z,4027.0
3148,6.0,30.0,30.0,10019.0,10019.0,5010.0,4868.0,142.0,2.834331,5010.0,county,10019.0,Yuma_Colorado_UnitedStates,Yuma,EP_LOC,Yuma_Colorado_UnitedStates,2020-11-14T00:00:00Z,8125.0
3149,NaN,NaN,NaN,14179.0,14179.0,4947.0,4268.0,679.0,13.725490,4947.0,county,14179.0,Zapata_Texas_UnitedStates,Zapata,EP_LOC,Zapata_Texas_UnitedStates,2020-11-14T00:00:00Z,48505.0
3150,NaN,NaN,NaN,11840.0,11840.0,3309.0,2748.0,561.0,16.953762,3309.0,county,11840.0,Zavala_Texas_UnitedStates,Zavala,EP_LOC,Zavala_Texas_UnitedStates,2020-11-14T00:00:00Z,48507.0


## Are there Null Fips Codes in the C3.ai API Call? We would need to drop them; assuming their information would not show up in the case counts either

There are 3138 unique fips codes so we will need to drop any null values in the "fips.id" column

In [113]:
df['fips.id'].nunique()

3138

# All Counties FIPS

In [65]:
state_dict = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}

In [111]:
# csv file from the c3.ai documentation
all_counties = pd.read_csv('c3ai-datalake-Location-Id-keys.csv', header = [2]).iloc[:,:4]
us_counties = all_counties.loc[(all_counties['Country'] == 'United States') 
                               & (all_counties['State'].isin(state_dict.keys()))
                                  & (all_counties['County'].str.contains('Out of')==False)
                              & (all_counties['County'].str.contains('Unassigned')==False)]
us_counties

,County id,County,State,Country
0,Autauga_Alabama_UnitedStates,Autauga,Alabama,United States
1,Baldwin_Alabama_UnitedStates,Baldwin,Alabama,United States
2,Barbour_Alabama_UnitedStates,Barbour,Alabama,United States
3,Bibb_Alabama_UnitedStates,Bibb,Alabama,United States
4,Blount_Alabama_UnitedStates,Blount,Alabama,United States
...,...,...,...,...
3143,Uinta_Wyoming_UnitedStates,Uinta,Wyoming,United States
3144,Washakie_Wyoming_UnitedStates,Washakie,Wyoming,United States
3145,Weston_Wyoming_UnitedStates,Weston,Wyoming,United States
3146,DukesandNantucket_Massachusetts_UnitedStates,Dukes and Nantucket,Massachusetts,United States


In [160]:
us_df = us_counties.merge(df[['fips.id','id']], left_on = 'County id', right_on = 'id')
us_df = us_df[us_df['fips.id'].isna() == False]
us_df

,County id,County,State,Country,fips.id,id
0,Autauga_Alabama_UnitedStates,Autauga,Alabama,United States,1001.0,Autauga_Alabama_UnitedStates
1,Baldwin_Alabama_UnitedStates,Baldwin,Alabama,United States,1003.0,Baldwin_Alabama_UnitedStates
2,Barbour_Alabama_UnitedStates,Barbour,Alabama,United States,1005.0,Barbour_Alabama_UnitedStates
3,Bibb_Alabama_UnitedStates,Bibb,Alabama,United States,1007.0,Bibb_Alabama_UnitedStates
4,Blount_Alabama_UnitedStates,Blount,Alabama,United States,1009.0,Blount_Alabama_UnitedStates
...,...,...,...,...,...,...
3137,Sweetwater_Wyoming_UnitedStates,Sweetwater,Wyoming,United States,56037.0,Sweetwater_Wyoming_UnitedStates
3138,Teton_Wyoming_UnitedStates,Teton,Wyoming,United States,56039.0,Teton_Wyoming_UnitedStates
3139,Uinta_Wyoming_UnitedStates,Uinta,Wyoming,United States,56041.0,Uinta_Wyoming_UnitedStates
3140,Washakie_Wyoming_UnitedStates,Washakie,Wyoming,United States,56043.0,Washakie_Wyoming_UnitedStates


## There a NA values for various values in our dataset mostly for Alaska and Hawaii. Dropped all Hawaii and Alaska counties because they are isolated away from mainland

In [161]:
for i in us_counties['County id']:
    if i not in list(us_df['County id']):
        print(i)

Kusilvak_Alaska_UnitedStates
Skagway_Alaska_UnitedStates
Kalawao_Hawaii_UnitedStates
St.LouisCity_Missouri_UnitedStates
NewYorkCity_NewYork_UnitedStates
KingandSnohomish_Washington_UnitedStates
Kirkland_Washington_UnitedStates
OtherCounties_Washington_UnitedStates
DukesandNantucket_Massachusetts_UnitedStates
KansasCity_Missouri_UnitedStates


# All fips will have no Counties in Alaska or Hawaii

In [172]:
AKHI_indx = us_df[(us_df['State'] == 'Alaska') | (us_df['State'] == 'Hawaii')].index
mainland = us_df.drop(AKHI_indx)
mainland

,County id,County,State,Country,fips.id,id
0,Autauga_Alabama_UnitedStates,Autauga,Alabama,United States,1001.0,Autauga_Alabama_UnitedStates
1,Baldwin_Alabama_UnitedStates,Baldwin,Alabama,United States,1003.0,Baldwin_Alabama_UnitedStates
2,Barbour_Alabama_UnitedStates,Barbour,Alabama,United States,1005.0,Barbour_Alabama_UnitedStates
3,Bibb_Alabama_UnitedStates,Bibb,Alabama,United States,1007.0,Bibb_Alabama_UnitedStates
4,Blount_Alabama_UnitedStates,Blount,Alabama,United States,1009.0,Blount_Alabama_UnitedStates
...,...,...,...,...,...,...
3137,Sweetwater_Wyoming_UnitedStates,Sweetwater,Wyoming,United States,56037.0,Sweetwater_Wyoming_UnitedStates
3138,Teton_Wyoming_UnitedStates,Teton,Wyoming,United States,56039.0,Teton_Wyoming_UnitedStates
3139,Uinta_Wyoming_UnitedStates,Uinta,Wyoming,United States,56041.0,Uinta_Wyoming_UnitedStates
3140,Washakie_Wyoming_UnitedStates,Washakie,Wyoming,United States,56043.0,Washakie_Wyoming_UnitedStates


In [176]:
mainland_fips = pd.DataFrame({"fips_code":mainland['fips.id'], "county": mainland['County'], "state": mainland['State'], "c3_data_id": mainland['id']})
mainland_fips.to_csv('./all_counties_model/mainland_counties_fips.csv', index = False)

# Note: There are NO Missing "Counties" in either mainland or full us df

- NYC: considered a borough with 5 separate counties (all five are in the dataframe):
    * Bronx
    * Kings
    * New York 
    * Queens
    * Richmond
- King and Snohomish Counties are separate Counties in Washington
    * Kirkland Washington is a City in King County, Washington (above)
- Dukes and Nantucket are separate counties in MA
- Kansas City is a city in Missouri in Wyandotte County, Kansas

In [174]:
us_df[us_df['County id'].isin(["Bronx_NewYork_UnitedStates","Kings_NewYork_UnitedStates",
                            "NewYork_NewYork_UnitedStates","Queens_NewYork_UnitedStates",
                            "Richmond_NewYork_UnitedStates"])]

,County id,County,State,Country,fips.id,id
1830,Bronx_NewYork_UnitedStates,Bronx,New York,United States,36005.0,Bronx_NewYork_UnitedStates
1851,Kings_NewYork_UnitedStates,Kings,New York,United States,36047.0,Kings_NewYork_UnitedStates
1858,NewYork_NewYork_UnitedStates,New York City,New York,United States,36061.0,NewYork_NewYork_UnitedStates
1868,Queens_NewYork_UnitedStates,Queens,New York,United States,36081.0,Queens_NewYork_UnitedStates
1870,Richmond_NewYork_UnitedStates,Richmond,New York,United States,36085.0,Richmond_NewYork_UnitedStates


In [133]:
us_df[us_df['County id'].isin(["King_Washington_UnitedStates","Snohomish_Washington_UnitedStates"])]

,County id,County,State,Country,fips.id,id
2969,King_Washington_UnitedStates,King,Washington,United States,53033.0,King_Washington_UnitedStates
2983,Snohomish_Washington_UnitedStates,Snohomish,Washington,United States,53061.0,Snohomish_Washington_UnitedStates


In [134]:
us_df[us_df['County id'].isin(["Dukes_Massachusetts_UnitedStates","Nantucket_Massachusetts_UnitedStates"])]

,County id,County,State,Country,fips.id,id
1220,Dukes_Massachusetts_UnitedStates,Dukes,Massachusetts,United States,25007.0,Dukes_Massachusetts_UnitedStates
1226,Nantucket_Massachusetts_UnitedStates,Nantucket,Massachusetts,United States,25019.0,Nantucket_Massachusetts_UnitedStates


In [135]:
us_df[us_df['County id'].isin(["Wyandotte_Kansas_UnitedStates"])]

,County id,County,State,Country,fips.id,id
992,Wyandotte_Kansas_UnitedStates,Wyandotte,Kansas,United States,20209.0,Wyandotte_Kansas_UnitedStates


In [136]:
all_fips = pd.DataFrame({"fips_code":us_df['fips.id'], "county": us_df['County'], "state": us_df['State'], "c3_data_id": us_df['id']})
all_fips

,fips_code,county,state,c3_data_id
0,1001.0,Autauga,Alabama,Autauga_Alabama_UnitedStates
1,1003.0,Baldwin,Alabama,Baldwin_Alabama_UnitedStates
2,1005.0,Barbour,Alabama,Barbour_Alabama_UnitedStates
3,1007.0,Bibb,Alabama,Bibb_Alabama_UnitedStates
4,1009.0,Blount,Alabama,Blount_Alabama_UnitedStates
...,...,...,...,...
3137,56037.0,Sweetwater,Wyoming,Sweetwater_Wyoming_UnitedStates
3138,56039.0,Teton,Wyoming,Teton_Wyoming_UnitedStates
3139,56041.0,Uinta,Wyoming,Uinta_Wyoming_UnitedStates
3140,56043.0,Washakie,Wyoming,Washakie_Wyoming_UnitedStates


In [177]:
all_fips.to_csv('./all_counties_model/all_counties_fips.csv', index = False)